# Azure OpenAI Chat Client Setup
This notebook walks through configuring and calling Azure OpenAI using the .NET SDK. Work through the cells in order to authenticate, configure, create, and execute a chat request.

## Workflow Overview
1. Load required namespaces for Azure client libraries.
2. Read configuration values for the model and endpoint from `appsettings.json`.
3. Build an authenticated `ChatClient` using `DefaultAzureCredential`.
4. Send a sample chat request and print the assistant response.

In [ ]:
// Step 1: Import required namespaces for Azure OpenAI access
using Azure;
using Azure.Identity;
using Azure.Core;
using Azure.Core.Pipeline;
using Azure.AI.OpenAI;
using Azure.AI.Projects;
using Azure.AI.Inference;
using OpenAI.Chat;
using System.ClientModel.Primitives;
using OpenAI;
using Microsoft.Extensions.Configuration;

### Step 1 Notes
Ensure your project references the Azure OpenAI and identity packages so the `using` directives resolve. Run this cell once per session to compile the script.

In [ ]:
// Step 2: Load configuration settings from appsettings.json
#pragma warning disable OPENAI001 // currently required for token based authentication

IConfiguration configuration = new ConfigurationBuilder()
    .AddJsonFile("appsettings.json")
    .Build();

var model = configuration["Model"];
var endpoint = configuration["AzureOpenAIEndpoint"];

### Step 2 Notes
Update `appsettings.json` with `Model` and `AzureOpenAIEndpoint` keys. The endpoint must match your deployed Azure OpenAI resource URL.

In [ ]:
// Step 3: Create an authenticated ChatClient for Azure OpenAI access
BearerTokenPolicy tokenPolicy = new(
    new DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default");

ChatClient client = new(
    model: model,
    authenticationPolicy: tokenPolicy,
    options: new OpenAIClientOptions()
    {
        Endpoint = new Uri(endpoint + "/openai/v1")
    }
);

ChatCompletionOptions options = new ChatCompletionOptions
{
    ReasoningEffortLevel = ChatReasoningEffortLevel.Low,
    MaxOutputTokenCount = 100000
};

### Step 3 Notes
The `BearerTokenPolicy` uses Azure AD credentials. Make sure your identity has `Cognitive Services OpenAI User` role on the resource.

In [ ]:
// Step 4: Execute the chat completion request and display the assistant response
ChatCompletion completion = client.CompleteChat(
    new DeveloperChatMessage("You are a helpful assistant that knows about gym, workouts, and fitness."),
    new UserChatMessage("What kinds of protein do you know about? Just a short list please.")
);

Console.WriteLine($"[ASSISTANT]: {completion.Content[0].Text}");

### Step 4 Notes
Modify the system and user prompts to fit your scenario. Inspect `completion` for reasons metadata if you need to trace filtering or tool calls.